In [12]:
import pandas as pd
import numpy as np

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import pickle

# -----------------------------
# File path
# -----------------------------
file_path = "cleaned_wells_final.xlsx"
excel = pd.ExcelFile(file_path)

# -----------------------------
# Column mapping
# -----------------------------
column_map = {
    "Depth": "Depth",
    "Depth(m)": "Depth",
    "Bit Weight(klb)": "WOB",
    "Rotary RPM(RPM)": "RPM",
    "RPM(RPM)": "RPM",
    "Flow In Rate(galUS/min)": "Flow",
    "ROP - Average(m/hr)": "ROP"
}

# -----------------------------
# Read all sheets and clean
# -----------------------------
all_wells = []

for sheet in excel.sheet_names:
    df = pd.read_excel(file_path, sheet_name=sheet)
    df.columns = df.columns.str.strip()

    # Rename columns
    df = df.rename(columns={c: column_map[c] for c in df.columns if c in column_map})

    # Keep only required columns
    required = ["Depth", "WOB", "RPM", "Flow", "ROP"]
    if not all(col in df.columns for col in required):
        continue

    df = df[required].dropna()
    df = df[(df > 0).all(axis=1)]  # remove negative / zero values

    # Optional: store well name
    df["Well"] = sheet

    all_wells.append(df)

# Combine all wells into one dataframe
global_df = pd.concat(all_wells, ignore_index=True)
print("Total samples across all wells:", len(global_df))

Total samples across all wells: 7292


In [15]:
# Features + target
X = global_df[["Depth", "WOB", "RPM", "Flow"]]
y = global_df["ROP"]

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Random Forest
rf_model = RandomForestRegressor(
    n_estimators=400,
    max_depth=None,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

# Train model
rf_model.fit(X_train, y_train)

# -----------------------------
# Evaluate
# -----------------------------
y_pred = rf_model.predict(X_test)

r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"R² Score : {r2:.4f}")
print(f"MSE      : {mse:.4f}")
print(f"MAE      : {mae:.4f}")

# -----------------------------
# Save model
# -----------------------------
with open("rf_model_depth_global.pkl", "wb") as f:
    pickle.dump(rf_model, f)

print("✅ Model saved as rf_model_depth_global.pkl")

R² Score : 0.5420
MSE      : 65.4646
MAE      : 3.2204
✅ Model saved as rf_model_depth_global.pkl


In [16]:
single_test = pd.DataFrame({
    "Depth": [1405],
    "WOB": [8.5],
    "RPM": [67.2],
    "Flow": [672]
})

predicted_rop = rf_model.predict(single_test)[0]
print("Predicted ROP:", round(predicted_rop, 2), "m/hr")

Predicted ROP: 10.62 m/hr
